In [1]:
from maze_env import *
import sys
import pygame 
from pygame.locals import *
import numpy as np
from create_maze import maze
import time
import shelve

pygame 2.5.2 (SDL 2.28.3, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
height=5
width=5
start_pos=(2,2)
goal_pos=(height*2,width*2)

In [3]:
#迷路ぱらめーた(高さ,横)
m=maze(height,width)
m.create(start_pos[0],start_pos[1])
m.set_goal_pos(goal_pos[0],goal_pos[1])
m.set_player(start_pos[0],start_pos[1])
m.break_wall(2)
m.Panel


7 8
3 6


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 2., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 3., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
# 何回ゴールするか
EPISODE_MAX = 1000
# ゴールまでの打ち切りステップ数
STEP_MAX = 2000
# 学習率
LEARNING_RATE = 0.1
# 割引率
DISCOUNT_RATE = 0.95
# 描画スピード
SLEEP_TIME = 0.001
q_learn = Q_learning(m.Panel,LEARNING_RATE,DISCOUNT_RATE) 


10


In [11]:
for episode in range(EPISODE_MAX):
    step = 0
    m.reset()
    # ランダムに最善でない行動を取る
    random_rate = 0.01 + 0.9 / (1 + episode)
    while not m.clear and step < STEP_MAX:
                # エージェントの1ステップ(行動、評価値の更新)
        # 行動の選択。ベストアクションとは限らない。
        action=q_learn.select_action(random_rate)
        # 選択された行動に従い動く。ただし、壁がある場合は無視される
        m.move(action)
        # 移動後の状態を取得
        next_state=find(m.Panel,2)
        if q_learn.state==next_state:
            pass
        else:
            step += 1

        # ベストアクションを選択
        next_action = q_learn.Q[:,q_learn.state[0],q_learn.state[1]].argmax()
        # Q[s][a] += 学習率 * ( 報酬 + 割引率 * ( max_{s'} Q[s'][a'] ) - Q[s][a] )aaaa
        q_learn.Q[action][q_learn.state[0]][q_learn.state[1]]+=q_learn.learning_rate*(q_learn.reward(m.clear)+
                                                                          q_learn.discount_rate*q_learn.Q[next_action][next_state[0]][next_state[1]]-
                                                                          q_learn.Q[action][q_learn.state[0]][q_learn.state[1]])
        # 移動後の状態を現在の状態に記録
        q_learn.state = next_state
        # 迷路描

    if episode%100==0:
        print(f"episode : {episode} step : {step} ")

episode : 0 step : 229 
episode : 100 step : 24 
episode : 200 step : 26 
episode : 300 step : 28 
episode : 400 step : 27 
episode : 500 step : 26 
episode : 600 step : 24 
episode : 700 step : 24 
episode : 800 step : 40 
episode : 900 step : 24 


In [6]:
from pygame.locals import *

In [12]:
panel_size=15
step=0
m.reset()
#画面描く
pygame.init()
screen=pygame.display.set_mode((800,600))
#タイトル
pygame.display.set_caption("迷路")
surface=pygame.Surface(screen.get_size(),pygame.HWACCEL)
font=pygame.font.SysFont("Arial",50)

while(True):
    #操作するもの
    
    screen.blit(surface,(0,0))
    
    #枠組み(10,20)ブロック
    #1ブロックあたり20ピクセル
    #色をつける。
    for i in range(m.Panel.shape[0]):
        for j in range(m.Panel.shape[1]):
            if m.Panel[i][j]==0:
                color=(0,0,0)
            if m.Panel[i][j]==1:
                color=(255,255,255)
            if m.Panel[i][j]==2:
                color=(100,100,0)
            if m.Panel[i][j]==3:
                color=(100,0,100)
            if m.Panel[i][j]==-1:
                color=(0,50,0)
            pygame.draw.rect(surface,color,(panel_size*j+panel_size,panel_size*i+panel_size,panel_size,panel_size))
    #ステップ表示
    text=font.render(str(step),True,(0,0,200))
    surface.blit(text,(10,10))
    #ゴールした場合
    if m.clear==True:
        text=font.render("CLEAR!  STEP:"+str(step),True,(250,250,0))
        surface.blit(text,(200,300))

    pygame.time.wait(10)
    pygame.display.update()

    #操作キー
    for event in pygame.event.get():
        if event.type==QUIT:
            pygame.quit()
            sys.exit()
        if event.type==KEYDOWN:
            if event.key==K_ESCAPE:
                pygame.quit()
                sys.exit()
    #移動キー
    action=q_learn.select_action(random_rate)
    time.sleep(0.2)
    if m.clear==False:
        m.move(action)
        # 移動後の状態を取得
        next_state=find(m.Panel,2)
        if q_learn.state==next_state:
            pass
        else:
            step+=1

        # ベストアクションを選択
        next_action = q_learn.Q[:,q_learn.state[0],q_learn.state[1]].argmax()
        # Q[s][a] += 学習率 * ( 報酬 + 割引率 * ( max_{s'} Q[s'][a'] ) - Q[s][a] )aaaa
        q_learn.Q[action][q_learn.state[0]][q_learn.state[1]]+=q_learn.learning_rate*(q_learn.reward(m.clear)+
                                                                          q_learn.discount_rate*q_learn.Q[next_action][next_state[0]][next_state[1]]-
                                                                          q_learn.Q[action][q_learn.state[0]][q_learn.state[1]])
        # 移動後の状態を現在の状態に記録
        q_learn.state = next_state

SystemExit: 

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.],
       [0., 1., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0., 3., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.